# Fed-BioMed Researcher GPU usage example

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

This example demonstrates using a Nvidia GPU for training a model.

The nodes for this example need to run on a machine providing a Nvidia GPU with enough GPU memory (and from a not-too-old model, so that it is supported by PyTorch).

If GPU doesn't have enough memory you will get a **out of memory error** at run time.

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the nodes up
We need at least 1 node, let's test using 3 nodes. For each node, add the MNIST dataset :
1. `./scripts/fedbiomed_run node config config1.ini add`, `./scripts/fedbiomed_run node config config2.ini add`, `./scripts/fedbiomed_run node config config3.ini add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node config config1.ini list`, `./scripts/fedbiomed_run node config config2.ini list`, `./scripts/fedbiomed_run node config config3.ini list`
3. Run the first node using `./scripts/fedbiomed_run node config config1.ini start --gpu` so that the nodes offers GPU for training and uses the default device.
4. Run the second node using `./scripts/fedbiomed_run node config config2.ini start --gpunum 1` so that the nodes offers GPU for training and requests using the 2nd GPU (device 1) but will fallback to default device if you don't have 2 GPUs on this machine.
5. Run the third node using `./scripts/fedbiomed_run node config config3.ini start` so that the nodes doesn't offers GPU for training (default behaviour).
6. Wait until you get `Starting task manager` for each node, it means you are online.

## Define an experiment model and parameters"

Declare a torch training plan MyTrainingPlan class to send for training on the node

In [2]:
import os

import numpy as np
import torch
import torch.nn as nn
from monai.networks.nets import UNet
from torch.optim import AdamW
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torch.utils.data import Dataset
import torchio as tio
import torchmetrics


class SarcoTrainingPlan(TorchTrainingPlan):
    
    # Defining model
    class Net(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.model = UNet(**model_args)

        def forward(self, x):
            output = self.model(x)
            return output
    

    # Defining loss
    
    class RobustCrossEntropyLoss(nn.CrossEntropyLoss):
        """
        this is just a compatibility layer because my target tensor is float and has an extra dimension
        """
        def forward(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
            if len(target.shape) == len(input.shape):
                assert target.shape[1] == 1
                target = target[:, 0]
            return super().forward(input, target.long())
    
    
    
    class SoftDiceLoss(nn.Module):
        def __init__(self, apply_nonlin=None, batch_dice=False, do_bg=True, smooth=1.):
            """
            """
            super().__init__()

            self.do_bg = do_bg
            self.batch_dice = batch_dice
            self.apply_nonlin = apply_nonlin
            self.smooth = smooth

        def forward(self, x, y, loss_mask=None):
            def sum_tensor(inp, axes, keepdim=False):
                axes = np.unique(axes).astype(int)
                if keepdim:
                    for ax in axes:
                        inp = inp.sum(int(ax), keepdim=True)
                else:
                    for ax in sorted(axes, reverse=True):
                        inp = inp.sum(int(ax))
                return inp


            def get_tp_fp_fn_tn(net_output, gt, axes=None, mask=None, square=False):
                """
                net_output must be (b, c, x, y(, z)))
                gt must be a label map (shape (b, 1, x, y(, z)) OR shape (b, x, y(, z))) or one hot encoding (b, c, x, y(, z))
                if mask is provided it must have shape (b, 1, x, y(, z)))
                :param net_output:
                :param gt:
                :param axes: can be (, ) = no summation
                :param mask: mask must be 1 for valid pixels and 0 for invalid pixels
                :param square: if True then fp, tp and fn will be squared before summation
                :return:
                """
                if axes is None:
                    axes = tuple(range(2, len(net_output.size())))

                shp_x = net_output.shape
                shp_y = gt.shape

                with torch.no_grad():
                    if len(shp_x) != len(shp_y):
                        gt = gt.view((shp_y[0], 1, *shp_y[1:]))

                    if all([i == j for i, j in zip(net_output.shape, gt.shape)]):
                        # if this is the case then gt is probably already a one hot encoding
                        y_onehot = gt
                    else:
                        gt = gt.long()
                        y_onehot = torch.zeros(shp_x)
                        if net_output.device.type == "cuda":
                            y_onehot = y_onehot.cuda(net_output.device.index)
                        y_onehot.scatter_(1, gt, 1)

                tp = net_output * y_onehot
                fp = net_output * (1 - y_onehot)
                fn = (1 - net_output) * y_onehot
                tn = (1 - net_output) * (1 - y_onehot)

                if mask is not None:
                    tp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(tp, dim=1)), dim=1)
                    fp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fp, dim=1)), dim=1)
                    fn = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fn, dim=1)), dim=1)
                    tn = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(tn, dim=1)), dim=1)

                if square:
                    tp = tp ** 2
                    fp = fp ** 2
                    fn = fn ** 2
                    tn = tn ** 2

                if len(axes) > 0:
                    tp = sum_tensor(tp, axes, keepdim=False)
                    fp = sum_tensor(fp, axes, keepdim=False)
                    fn = sum_tensor(fn, axes, keepdim=False)
                    tn = sum_tensor(tn, axes, keepdim=False)

                return tp, fp, fn, tn


            shp_x = x.shape

            if self.batch_dice:
                axes = [0] + list(range(2, len(shp_x)))
            else:
                axes = list(range(2, len(shp_x)))

            if self.apply_nonlin is not None:
                x = self.apply_nonlin(x)

            tp, fp, fn, _ = get_tp_fp_fn_tn(x, y, axes, loss_mask, False)

            nominator = 2 * tp + self.smooth
            denominator = 2 * tp + fp + fn + self.smooth

            dc = nominator / (denominator + 1e-8)

            if not self.do_bg:
                if self.batch_dice:
                    dc = dc[1:]
                else:
                    dc = dc[:, 1:]
            dc = dc.mean()

            return -dc
    

    class DC_and_CE_loss(nn.Module):
        def __init__(self, soft_dice_kwargs, ce_kwargs, aggregate="sum", square_dice=False, weight_ce=1, weight_dice=1,
                     log_dice=False, ignore_label=None):
            """
            CAREFUL. Weights for CE and Dice do not need to sum to one. You can set whatever you want.
            :param soft_dice_kwargs:
            :param ce_kwargs:
            :param aggregate:
            :param square_dice:
            :param weight_ce:
            :param weight_dice:
            """
            super().__init__()
            if ignore_label is not None:
                assert not square_dice, 'not implemented'
                ce_kwargs['reduction'] = 'none'
            self.log_dice = log_dice
            self.weight_dice = weight_dice
            self.weight_ce = weight_ce
            self.aggregate = aggregate
            self.ce = SarcoTrainingPlan.RobustCrossEntropyLoss(**ce_kwargs)

            self.ignore_label = ignore_label

            self.dc = SarcoTrainingPlan.SoftDiceLoss(apply_nonlin=lambda x: torch.softmax(x, dim=1), **soft_dice_kwargs)

        def forward(self, net_output, target):
            """
            target must be b, c, x, y(, z) with c=1
            :param net_output:
            :param target:
            :return:
            """
            if self.ignore_label is not None:
                assert target.shape[1] == 1, 'not implemented for one hot encoding'
                mask = target != self.ignore_label
                target[~mask] = 0
                mask = mask.float()
            else:
                mask = None

            dc_loss = self.dc(net_output, target, loss_mask=mask) if self.weight_dice != 0 else 0
            if self.log_dice:
                dc_loss = -torch.log(-dc_loss)

            ce_loss = self.ce(net_output, target[:, 0].long()) if self.weight_ce != 0 else 0
            if self.ignore_label is not None:
                ce_loss *= mask[:, 0]
                ce_loss = ce_loss.sum() / mask.sum()

            if self.aggregate == "sum":
                result = self.weight_ce * ce_loss + self.weight_dice * dc_loss
            else:
                raise NotImplementedError("nah son") # reserved for other stuff (later)
            return result
    
    class SarcoDataset(Dataset):
        def __init__(self, csv_path, transform=None, nb_samples=8):
            self.root_dir = os.path.dirname(csv_path)
            self.nb_samples = nb_samples

            def numpy_reader2D_no_affine(path):
                data = np.load(path)
                data = data[:, :, 0]
                return data, np.eye(4)

            subjects_list = []
            for patient in sorted(os.listdir(self.root_dir)):
                try:
                    subj = tio.Subject(ct=tio.ScalarImage(path=os.path.join(self.root_dir, patient, "ct.npy"),
                                                          reader=numpy_reader2D_no_affine),
                                       mask=tio.LabelMap(path=os.path.join(self.root_dir, patient, "mask.npy"),
                                                         reader=numpy_reader2D_no_affine),
                                       )
                    subj.load()
                    subjects_list.append(subj)
                except Exception:
                    print(f" {patient} failed")
            subjects_dataset = tio.SubjectsDataset(subjects_list, transform=transform)
            patch_size = (512, 512)
            sampler = tio.data.UniformSampler((512, 512, 1))
            
            self.patches_queue = tio.Queue(subjects_dataset, max_length=128 , samples_per_volume=8, sampler=sampler,
                                shuffle_patches=True, shuffle_subjects=True)

        def __len__(self):
            return len(self.patches_queue)

        def __getitem__(self, idx):
            
            if torch.is_tensor(idx):
                idx = idx.tolist()
                
            inputs = torch.squeeze(self.patches_queue[idx]["ct"][tio.DATA], dim=-1).float()
            target = torch.squeeze(self.patches_queue[idx]["mask"][tio.DATA], dim=-1)

            return inputs.float(), target.long()

    def init_model(self, model_args):
        self.criterion = self.DC_and_CE_loss({'batch_dice': True, 'smooth': 1e-5, 'do_bg': False}, {})
        # self.criterion = torch.nn.NLLLoss()
        self.metric = torchmetrics.Dice(num_classes=4, ignore_index=0)
        self.mean_dsc = .0
        return self.Net(model_args=model_args)

    def init_optimizer(self, optimizer_args):
        self._optimizer = AdamW(self.model().parameters(), lr=optimizer_args["lr"], weight_decay=1e-3)
        self.sgdr = torch.optim.lr_scheduler.ExponentialLR(self._optimizer, 0.9549925860)
        return self._optimizer

    def init_dependencies(self):
        return [
            "import os",
            "import numpy as np",
            "import torch",
            "import torch.nn as nn",
            "from monai.networks.nets import UNet",
            "from torch.optim import AdamW",
            "from fedbiomed.common.training_plans import TorchTrainingPlan",
            "from fedbiomed.common.data import DataManager",
            "from torch.utils.data import Dataset",
            "import torchio as tio",
            "import torchmetrics"
        ]

    def training_data(self, batch_size=16):
        augments = tio.Compose([
            tio.CropOrPad((512, 512, 1)),
            tio.RandomAffine(scales=(0.85, 1.25), degrees=(0, 0, 0, 0, -15, 15), isotropic=True, p=.2),
            tio.RandomNoise(std=(0, 0.1), p=.1),
            tio.RandomBlur(std=(.5, 1.), p=.2)
        ])

        dataset1 = self.SarcoDataset(csv_path=self.dataset_path, transform=augments)
        loader_arguments = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset1, **loader_arguments)

    def training_step(self, data, target):
        output = self.model().forward(data)
        loss = self.criterion(output, target)
        dsc = self.metric.to(self._device)(torch.softmax(output, dim=1), target.long()).item()
        self.mean_dsc += dsc
        return loss
  
    def _train_over_batch(self, data, target):
        """Train the model over a single batch of data.

        This function handles all the torch-specific logic concerning model training, including backward propagation,
        aggregator-specific correction terms, and optimizer stepping.

        Args:
            data: the input data to the model
            target: the training labels

        Returns:
            corrected loss: the loss value used for backward propagation, including any correction terms
            loss: the uncorrected loss for reporting
        """
        # zero-out gradients
        self._optimizer.zero_grad()
        # FIXME: `self._optimizer.train()` is never called but should be. 
        # FIXME 2: Should we move training process to `Optimizer` or `Model` class?

        # compute loss
        loss = self.training_step(data, target)  # raises an exception if not provided
        corrected_loss = torch.clone(loss)

        # If FedProx is enabled: use regularized loss function
        if self._fedprox_mu is not None:
            corrected_loss += float(self._fedprox_mu) / 2 * self.__norm_l2()

        # Run the backward pass to compute parameters' gradients
        corrected_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model().parameters(), 12)

        # If Scaffold is used: apply corrections to the gradients
        if self.aggregator_name is not None and self.aggregator_name.lower() == "scaffold":
            for name, param in self.model().named_parameters():
                correction = self.correction_state.get(name)
                if correction is not None:
                    param.grad.sub_(correction.to(param.grad.device))

        # Have the optimizer collect, refine and apply gradients
        self._optimizer.step()
        self.sgdr.step()
        print(f"\rLearning rate: {self.sgdr.print_lr()}", end="")

        return corrected_loss, loss



# Defining silo BN aggregator

In [3]:
from typing import Dict, Union, Mapping

import torch # used by typing
import numpy # used by typing

from fedbiomed.common.constants import ErrorNumbers
from fedbiomed.common.exceptions import FedbiomedAggregatorError
from fedbiomed.researcher.aggregators.aggregator import Aggregator
from fedbiomed.researcher.aggregators.functional import federated_averaging


class SiloBN(Aggregator):
    """
    Defines the Federated averaging strategy
    """

    def __init__(self):
        """Construct `FedAverage` object as an instance of [`Aggregator`]
        [fedbiomed.researcher.aggregators.Aggregator].
        """
        super().__init__()
        self.aggregator_name = "FedAverage"

    def aggregate(
            self,
            model_params: Dict[str, Dict[str, Union['torch.Tensor', 'numpy.ndarray']]],
            weights: Dict[str, float],
            *args,
            **kwargs
    ) -> Mapping[str, Union['torch.Tensor', 'numpy.ndarray']]:
        """ Aggregates  local models sent by participating nodes into a global model, following Federated Averaging
        strategy.

        weights is a list of single-item dictionaries, each dictionary has the node id as key, and the weight as value.
        model_params is a list of single-item dictionaries, each dictionary has the node is as key,
        and a framework-specific representation of the model parameters as value.

        Args:
            model_params: contains each model layers
            weights: contains all weights of a given layer.

        Returns:
            Aggregated parameters
        """

        model_params_processed = []
        weights_processed = []

        for node_id, params in model_params.items():

            if node_id not in weights:
                raise FedbiomedAggregatorError(
                    f"{ErrorNumbers.FB401.value}. Can not find corresponding calculated weight for the "
                    f"node {node_id}. Aggregation is aborted."
                )
            if not "bn.running" in node_id and not "bn.num_batches_tracked" in node_id:
                weight = weights[node_id]
                model_params_processed.append(params)
                weights_processed.append(weight)

        if any([x < 0. or x > 1. for x in weights_processed]) or sum(weights_processed) == 0:
            raise FedbiomedAggregatorError(
                f"{ErrorNumbers.FB401.value}. Aggregation aborted due to sum of the weights is equal to 0 {weights}. "
                f"Sample sizes received from nodes might be corrupted."
            )

        agg_params = federated_averaging(model_params_processed, weights_processed)

        return agg_params

This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [4]:
model_args = {
    "spatial_dims": 2,
    "in_channels": 1,
    "out_channels": 4,
    "channels": (4, 8, 16, 32, 64),
    "strides": (2, 2, 2, 1),
}

training_args = {
    'batch_size': 16, 
    'optimizer_args': {
        'lr': 1e-3
    }, 
    'use_gpu': True,
    'epochs': 1, 
    'dry_run': False,  
#    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [5]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#sarco']
rounds = 100

exp = Experiment(
    tags=tags,
    model_args=model_args,
    training_plan_class=SarcoTrainingPlan,
    training_args=training_args,
    round_limit=rounds,
    aggregator=FedAverage(),
    node_selection_strategy=None,
    tensorboard=True,
)

2023-06-05 15:26:56,596 - Selected pickle protocol: '4'


2023-06-05 15:26:56,603 fedbiomed INFO - Messaging researcher_e177fe91-b52d-4fea-ae52-7cba97a6f5cf successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f04a42be550>
2023-06-05 15:26:56,641 fedbiomed INFO - Searching dataset with data tags: ['#sarco'] for all nodes
2023-06-05 15:27:03,201 fedbiomed INFO - INFO
					 NODE node_78684f14-a076-4642-9ce9-5424892fd7a3
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2023-06-05 15:27:06,648 fedbiomed INFO - Node selected for training -> node_78684f14-a076-4642-9ce9-5424892fd7a3
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
2023-06-05 15:27:06,662 fedbiomed DEBUG - Model file has been saved: /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/my_m

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [6]:
exp.set_training_args(training_args)
exp.run()

2023-06-05 15:27:06,723 fedbiomed INFO - Sampled nodes in round 0 ['node_78684f14-a076-4642-9ce9-5424892fd7a3']
2023-06-05 15:27:06,724 fedbiomed INFO - Sending request 
					 To: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_e177fe91-b52d-4fea-ae52-7cba97a6f5cf', 'job_id': '2a4c4d8f-f7cf-4984-8a03-46d6f7b36096', 'training_args': {'batch_size': 16, 'optimizer_args': {'lr': 0.001}, 'use_gpu': True, 'epochs': 1, 'dry_run': False, 'num_updates': None, 'batch_maxnum': None, 'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'log_interval': 10, 'fedprox_mu': None, 'dp_args': None}, 'training': True, 'model_args': {'spatial_dims': 2, 'in_channels': 1, 'out_channels': 4, 'channels': (4, 8, 16, 32, 64), 'strides': (2, 2, 2, 1)}, 'command': 'train', 'training_plan_url': 'http://192.168.106.2:8844/media/uploads/2023/06/05/my_model_75bd27b7

2023-06-05 15:27:31,908 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 2 Epoch: 1 | Iteration: 40/92 (43%) | Samples: 640/1472
 					 Loss: 1.008299 
					 ---------
2023-06-05 15:27:33,352 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 2 Epoch: 1 | Iteration: 50/92 (54%) | Samples: 800/1472
 					 Loss: 0.998159 
					 ---------
2023-06-05 15:27:34,433 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 2 Epoch: 1 | Iteration: 60/92 (65%) | Samples: 960/1472
 					 Loss: 1.006299 
					 ---------
2023-06-05 15:27:35,810 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 2 Epoch: 1 | Iteration: 70/92 (76%) | Samples: 1120/1472
 					 Loss: 1.008328 
					 ---------
2023-06-05 15:27:36,961 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 2 Epoch: 1 | I

2023-06-05 15:28:06,850 fedbiomed INFO - Saved aggregated params for round 2 in /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/aggregated_params9c51847c-e218-4312-9717-5586a24726e9.pt
2023-06-05 15:28:06,850 fedbiomed INFO - Sampled nodes in round 3 ['node_78684f14-a076-4642-9ce9-5424892fd7a3']
2023-06-05 15:28:06,851 fedbiomed INFO - Sending request 
					 To: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_e177fe91-b52d-4fea-ae52-7cba97a6f5cf', 'job_id': '2a4c4d8f-f7cf-4984-8a03-46d6f7b36096', 'training_args': {'batch_size': 16, 'optimizer_args': {'lr': 0.001}, 'use_gpu': True, 'epochs': 1, 'dry_run': False, 'num_updates': None, 'batch_maxnum': None, 'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'log_interval': 10, 'fedprox_mu': None, 'dp_args': None}, 'training': True, 'model_args': {'spatial_dims': 2, 'in

2023-06-05 15:28:30,904 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 5 Epoch: 1 | Iteration: 30/92 (33%) | Samples: 480/1472
 					 Loss: 0.713152 
					 ---------
2023-06-05 15:28:31,947 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 5 Epoch: 1 | Iteration: 40/92 (43%) | Samples: 640/1472
 					 Loss: 0.748097 
					 ---------
2023-06-05 15:28:33,320 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 5 Epoch: 1 | Iteration: 50/92 (54%) | Samples: 800/1472
 					 Loss: 0.666846 
					 ---------
2023-06-05 15:28:34,391 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 5 Epoch: 1 | Iteration: 60/92 (65%) | Samples: 960/1472
 					 Loss: 0.678848 
					 ---------
2023-06-05 15:28:35,673 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 5 Epoch: 1 | It

2023-06-05 15:29:06,959 fedbiomed DEBUG - upload (HTTP POST request) of file /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/aggregated_params23081123-4f38-44f3-b284-95bb0d1985cb.pt successful, with status code 201
2023-06-05 15:29:06,959 fedbiomed INFO - Saved aggregated params for round 5 in /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/aggregated_params23081123-4f38-44f3-b284-95bb0d1985cb.pt
2023-06-05 15:29:06,960 fedbiomed INFO - Sampled nodes in round 6 ['node_78684f14-a076-4642-9ce9-5424892fd7a3']
2023-06-05 15:29:06,961 fedbiomed INFO - Sending request 
					 To: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_e177fe91-b52d-4fea-ae52-7cba97a6f5cf', 'job_id': '2a4c4d8f-f7cf-4984-8a03-46d6f7b36096', 'training_args': {'batch_size': 16, 'optimizer_args': {'lr': 0.001}, 'use_gpu': True, 'epochs': 1, 'dry_run': False, 'num_updates': None, 'batch_maxnum': None, 'test_rati

...

2023-06-05 15:58:32,963 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 95 Epoch: 1 | Iteration: 10/92 (11%) | Samples: 160/1472
 					 Loss: 0.165044 
					 ---------
2023-06-05 15:58:34,136 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 95 Epoch: 1 | Iteration: 20/92 (22%) | Samples: 320/1472
 					 Loss: 0.157752 
					 ---------
2023-06-05 15:58:35,411 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 95 Epoch: 1 | Iteration: 30/92 (33%) | Samples: 480/1472
 					 Loss: 0.277779 
					 ---------
2023-06-05 15:58:36,490 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 95 Epoch: 1 | Iteration: 40/92 (43%) | Samples: 640/1472
 					 Loss: 0.256611 
					 ---------
2023-06-05 15:58:37,922 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 95 Epoch: 1

2023-06-05 15:59:11,188 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_abab3b33-4de9-4e99-99ea-f3506ed4c764.pt successful, with status code 200
2023-06-05 15:59:11,191 fedbiomed INFO - Nodes that successfully reply in round 95 ['node_78684f14-a076-4642-9ce9-5424892fd7a3']
2023-06-05 15:59:11,207 fedbiomed DEBUG - upload (HTTP POST request) of file /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/aggregated_params6a523fca-908a-4c54-98f3-1685ce3bbeb2.pt successful, with status code 201
2023-06-05 15:59:11,208 fedbiomed INFO - Saved aggregated params for round 95 in /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/aggregated_params6a523fca-908a-4c54-98f3-1685ce3bbeb2.pt
2023-06-05 15:59:11,208 fedbiomed INFO - Sampled nodes in round 96 ['node_78684f14-a076-4642-9ce9-5424892fd7a3']
2023-06-05 15:59:11,209 fedbiomed INFO - Sending request 
					 To: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Request: : Perform training with the argument

2023-06-05 15:59:33,121 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 98 Epoch: 1 | Iteration: 10/92 (11%) | Samples: 160/1472
 					 Loss: 0.254738 
					 ---------
2023-06-05 15:59:34,220 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 98 Epoch: 1 | Iteration: 20/92 (22%) | Samples: 320/1472
 					 Loss: 0.216481 
					 ---------
2023-06-05 15:59:35,577 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 98 Epoch: 1 | Iteration: 30/92 (33%) | Samples: 480/1472
 					 Loss: 0.341909 
					 ---------
2023-06-05 15:59:36,583 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 98 Epoch: 1 | Iteration: 40/92 (43%) | Samples: 640/1472
 					 Loss: 0.317409 
					 ---------
2023-06-05 15:59:37,991 fedbiomed INFO - TRAINING 
					 NODE_ID: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Round 98 Epoch: 1

2023-06-05 16:00:11,328 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_45f9b69b-4a5d-47f7-8a32-5f7c04dd3ab2.pt successful, with status code 200
2023-06-05 16:00:11,331 fedbiomed INFO - Nodes that successfully reply in round 98 ['node_78684f14-a076-4642-9ce9-5424892fd7a3']
2023-06-05 16:00:11,347 fedbiomed DEBUG - upload (HTTP POST request) of file /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/aggregated_params6cf40f45-b5b7-4a33-b9d4-11bc05ba9c3e.pt successful, with status code 201
2023-06-05 16:00:11,347 fedbiomed INFO - Saved aggregated params for round 98 in /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/aggregated_params6cf40f45-b5b7-4a33-b9d4-11bc05ba9c3e.pt
2023-06-05 16:00:11,348 fedbiomed INFO - Sampled nodes in round 99 ['node_78684f14-a076-4642-9ce9-5424892fd7a3']
2023-06-05 16:00:11,348 fedbiomed INFO - Sending request 
					 To: node_78684f14-a076-4642-9ce9-5424892fd7a3 
					 Request: : Perform training with the argument

100

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [7]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()


List the training rounds :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

List the nodes for the last training round and their timings : 
	- node_78684f14-a076-4642-9ce9-5424892fd7a3 :    
		rtime_training=11.06 seconds    
		ptime_training=82.73 seconds    
		rtime_total=20.02 seconds




,success,msg,dataset_id,node_id,params_path,params,optimizer_args,sample_size,timing
0,True,,dataset_e86ddf30-db34-4452-ae8b-90ae347c763f,node_78684f14-a076-4642-9ce9-5424892fd7a3,/home/natlap/fedbiomed-normandie/var/experimen...,{'model.model.0.conv.weight': [[tensor([[ 0.18...,{'lr': [1.4454397677610258e-05]},1464,"{'rtime_training': 11.060572979971766, 'ptime_..."


Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [8]:
print("\nList the training rounds : ", exp.aggregated_params().keys())
max_round = max(exp.aggregated_params().keys())
print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[max_round]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[max_round]['params'].keys())



List the training rounds :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

Access the federated params for the last training round :
	- params_path:  /home/natlap/fedbiomed-normandie/var/experiments/Experiment_0115/aggregated_paramsb8c2dd18-1730-479c-bd62-738cd284884c.pt
	- parameter data:  odict_keys(['model.model.0.conv.weight', 'model.model.0.conv.bias', 'model.model.0.adn.A.weight', 'model.model.1.submodule.0.conv.weight', 'model.model.1.submodule.0.conv.bias', 'model.model.1.submodule.0.adn.A.weight', 'model.model.1.submodule.1.submodule.0.conv.weight', 'model.model.1.submodule.1.submodule.0.conv.bias', 'model.model.1.submodule.1.

Feel free to run other sample notebooks or try your own models :D